In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

In [6]:
df_data = pd.read_csv('../data/train.csv')
df_data.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


## Génération des données d'entrainement et de test

In [7]:
X = df_data.copy().drop(['id', 'species'],axis='columns').values

species = df_data['species'].unique()

# Création des vecteurs one-hot pour la matrice de targets
t = np.zeros((len(X),species.size)) 
for i in range(species.size):
    t[df_data['species'] == species[i], i] = 1

Nous séparons en deux et de manièere aléatoire le jeu de données d'origine avec : 
- 70% qui constituerons les données d'entrainement;
- 30% qui constitueront les données de test;

De plus, nous avons dans notre dataset 99 espaces représentées chacune par 10 individus.
Si nous appliquons un ration 70% pour l'entrainement, 30% pour le test, nous aurons en moyenne :
- 7 individus par espèce pour les données d'entrainement
- 3 individus par espèce pour les données de test

In [8]:
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3, random_state=0)

Nous obtenons ainsi les résultat suivant :

In [9]:
print("size X_train: "+str(X_train.shape))
print("size t_train: "+str(t_train.shape))
print()
print("size X_test: "+str(X_test.shape))
print("size t_test: "+str(t_test.shape))

size X_train: (693, 192)
size t_train: (693, 99)

size X_test: (297, 192)
size t_test: (297, 99)


## Déclaration du model

In [3]:
np.logspace(-9, 1, 10)

array([1.00000000e-09, 1.29154967e-08, 1.66810054e-07, 2.15443469e-06,
       2.78255940e-05, 3.59381366e-04, 4.64158883e-03, 5.99484250e-02,
       7.74263683e-01, 1.00000000e+01])

In [64]:
model = MLPClassifier(random_state=1, max_iter=3000, hidden_layer_sizes=(96, 96), verbose=True, tol=1e-3)
model.fit(X_train, t_train)
print("précision de "+str(model.score(X_train, t_train)*100)+"%")

Iteration 1, loss = 68.16546828
Iteration 2, loss = 66.50320911
Iteration 3, loss = 64.57367223
Iteration 4, loss = 62.15856260
Iteration 5, loss = 59.03053164
Iteration 6, loss = 55.00366726
Iteration 7, loss = 49.89910011
Iteration 8, loss = 43.68589204
Iteration 9, loss = 36.57410114
Iteration 10, loss = 29.04491949
Iteration 11, loss = 21.82944237
Iteration 12, loss = 15.73320820
Iteration 13, loss = 11.25614330
Iteration 14, loss = 8.41296853
Iteration 15, loss = 6.85222341
Iteration 16, loss = 6.11572532
Iteration 17, loss = 5.82434229
Iteration 18, loss = 5.74219276
Iteration 19, loss = 5.73722883
Iteration 20, loss = 5.74854772
Iteration 21, loss = 5.75254647
Iteration 22, loss = 5.74360158
Iteration 23, loss = 5.72444635
Iteration 24, loss = 5.69960214
Iteration 25, loss = 5.67436314
Iteration 26, loss = 5.65039137
Iteration 27, loss = 5.63101527
Iteration 28, loss = 5.61458979
Iteration 29, loss = 5.60256381
Iteration 30, loss = 5.59409295
Iteration 31, loss = 5.58744338
Iter

In [65]:
model.score(X_test, t_test)

0.7037037037037037

## Test du model

In [40]:
model.score(X_test, t_test)

0.7946127946127947

Le MLP via scikit-learn met beaucoup de temps pour converger ce qui va rendre difficile la recherche d'hyperparamètre. Pour avoir plus de contrôle avec le MLP que nous souhaitons mettre en place, nous allons passé à une implémentation du MLP via PyTorch ce qui permettra d'avoir plus de contrôle sur la création du classifier.